# [fileinput]通过重定向/管道/文件接受输入

在命令行中传递一个文件名给该脚本

In [8]:
%%file filein.py

import fileinput
import subprocess

# ===== sys ======
# python "...\filein.py" FileinputTest.txt
import sys
a=sys.argv[0]
b=sys.argv[1]

print("filename:",a)
print("param1:",b)

with fileinput.input(b) as f_input:
    for line in f_input:
        print(line, end='')

# ===== input =====
#file = input("請輸入...")
#print(file)
#
#with fileinput.input(file) as f_input:
#    for line in f_input:
#        print(line, end='')
        
# 暫停        
subprocess.call("pause",shell=True)

Overwriting filein.py


In [9]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))

# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'filein.py')
cmd = "python " + f'"{DIR}"'  + ' FileinputTest.txt' # sys.argv[1]

# 印出 cmd DIR
print(cmd,BASE_DIR,sep='\n')

#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

# 暫停(需在要執行的 py 上)
#subprocess.call("pause",shell=True)
#os.system("pause")

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理\filein.py" FileinputTest.txt
D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理


`fileinput.input()` 创建并返回一个 `FileInput` 类的实例。 

该实例除了拥有一些有用的帮助方法外，它还可被当做一个上下文管理器使用。 

通过将它作为一个上下文管理器使用，可以确保它不再使用时文件能自动关闭。

因此，整合起来，如果我们要写一个打印多个文件输出的脚本，那么我们需要在输出中包含文件名和行号

In [34]:
import fileinput

with fileinput.input('FileinputTest.txt') as f:
    for line in f:
        print(f.filename(), f.lineno(), line, end='') 

FileinputTest.txt 1 1: AAAAA
FileinputTest.txt 2 2: BBBBB
FileinputTest.txt 3 3: CCCCC
FileinputTest.txt 4 4: DDDDD


# [raise SystemExit('It failed!')]终止程序并给出错误信息

抛出一个 `SystemExit` 异常，使用错误消息作为参数。

例如：

```py
raise SystemExit('It failed!')
```

它会将消息在 `sys.stderr` 中打印，然后程序以状态码1退出

当你想要终止某个程序时，你可能会像下面这样写：

```py
import sys
sys.stderr.write('It failed!\n')
raise SystemExit(1)
```

如果你直接将消息作为参数传给 `SystemExit()` ，那么你可以省略其他步骤， 

比如 `import` 语句或将错误消息写入 `sys.stderr`

# [argparse]解析命令行选项

首先要创建一个 `ArgumentParser` 实例， 并使用 `add_argument()` 方法声明你想要支持的选项。 

在每个 `add_argument()` 调用中， `dest` 参数指定解析结果被指派给属性的名字。

`metavar` 参数被用来生成帮助信息。

`action` 参数指定跟属性对应的处理逻辑， 通常的值为 `store` ,被用来存储某个值或将多个参数值收集到一个列表中。 

---

`nargs='*'` 下面的参数收集所有剩余的命令行参数到一个列表中。

在本例中它被用来构造一个文件名列表：

```py
parser.add_argument(dest='filenames',metavar='filename', nargs='*')
```

---

`action='store_true'` 下面的参数根据参数是否存在来设置一个 `Boolean` 标志：

```py
parser.add_argument('-v', dest='verbose', action='store_true',
                    help='verbose mode')
```

---

`action='store'` 下面的参数接受一个单独值并将其存储为一个字符串：

```py
parser.add_argument('-o', dest='outfile', action='store',
                    help='output file')
```

---

`action='append'` 下面的参数说明允许某个参数重复出现多次，并将它们追加到一个列表中去。 

`required` 标志表示该参数至少要有一个。

`-p` 和 `--pat` 表示两个参数名形式都可使用。

```py
parser.add_argument('-p', '--pat',metavar='pattern', required=True,
                    dest='patterns', action='append',
                    help='text pattern to search for')
```

---

最后， `choices={'slow','fast'}` 下面的参数说明接受一个值，但是会将其和可能的选择值做比较，以检测其合法性：

```py
parser.add_argument('--speed', dest='speed', action='store',
                    choices={'slow','fast'}, default='slow',
                    help='search speed')
```

---

一旦参数选项被指定，你就可以执行 `parser.parse()` 方法了。 它会处理 `sys.argv` 的值并返回一个结果实例。 

每个参数值会被设置成该实例中` add_argument()` 方法的 `dest` 参数指定的属性值。


In [14]:
%%file search.py
'''
Hypothetical command-line tool for searching a collection of
files for one or more text patterns.
'''

import os

import argparse
parser = argparse.ArgumentParser(description='Search some files')

parser.add_argument(dest='filenames',metavar='filename', nargs='*')

# required=True 必填
parser.add_argument('-p', '--pat',metavar='pattern', required=True,
                    dest='patterns', action='append',
                    help='text pattern to search for')

parser.add_argument('-v', dest='verbose', action='store_true',
                    help='verbose mode')

parser.add_argument('-o', dest='outfile', action='store',
                    help='output file')

parser.add_argument('--speed', dest='speed', action='store',
                    choices={'slow','fast'}, default='slow',
                    help='search speed')

args = parser.parse_args()

# Output the collected arguments
print(args.filenames)
print(args.patterns)
print(args.verbose)
print(args.outfile)
print(args.speed)

os.system("pause")

Overwriting search.py


In [24]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))

# =============== cmd 參數 ================
# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'search.py')
cmd = "python " + f'"{DIR}"' + ' -v -p spam -p eggs foo.txt'
# ' -v -p spam --pat=eggs foo.txt bar.txt -o results --speed=fast'

# 印出 cmd DIR
print(cmd,BASE_DIR,sep='\n')

#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

# 暫停(需在要執行的 py 上)
#subprocess.call("pause",shell=True)
#os.system("pause")

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理\search.py" -v -p spam -p eggs foo.txt
D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理


```py
ArgumentParser(
    prog=None, # program 的名字，None 就檔名
    usage=None, # None 就根據你設定的參數產生相對應的說明
    description=None, # 描述
    epilog=None # 補充說明 ex: epilog="see the doc: https://..."
    )
```

位置參數 (positional argument)

    parser.add_argument(dest='filenames',metavar='filename', nargs='*')
    
    # nargs='*' 任意位置(無前綴)的參數
     foo.txt

選擇性參數 (optional argument) 根據前綴來指定

    parser.add_argument('-o', dest='outfile', action='store',
                    help='output file')
                    
    -o results

# [getpass]运行时弹出密码输入提示

`Python` 的 `getpass` 模块正是你所需要的。

你可以让你很轻松的弹出密码输入提示， 并且不会在用户终端回显密码。

下面是具体代码

In [35]:
import getpass

user = getpass.getuser() # 'po390' # C:\Users\po390>
passwd = getpass.getpass()

def svc_login( passwd):
    if  passwd == "123":
        return True
    else:
        return False

if svc_login( passwd):    
    print('Yay!')
else:
    print('Boo!')
    
print(user)

········
Yay!
po390


如果你想显示的弹出用户名输入提示，使用内置的 `input` 函数：

```py
user = input('Enter your username: ')
```

# [os.get_terminal_size()]获取终端的大小

使用 `os.get_terminal_size()` 函数来做到这一点。

In [37]:
import os
sz = os.get_terminal_size()

sz
sz.columns
sz.lines

os.terminal_size(columns=120, lines=30)

120

30

# [subprocess]{ windows 底下 bash 的命令沒有反應}执行外部命令并获取它的输出

和 `shell` 有關 ， 在 `windows` 底下 `bash` 的命令沒有反應，

且預設編碼非 `utf8` 輸出也有困難

## 補充

In [18]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))

# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'filein.py')
cmd = "python " + f'"{DIR}"'  + ' FileinputTest.txt' # sys.argv[1]

# 印出 cmd DIR
print(cmd,BASE_DIR,sep='\n')

#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subp = subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

# 暫停(需在要執行的 py 上)
#subprocess.call("pause",shell=True)
#os.system("pause")

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理\filein.py" FileinputTest.txt
D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理



**參考** [Python3 subprocess](https://www.runoob.com/w3cnote/python3-subprocess.html)

In [20]:
import subprocess
def runcmd(command):
    ret = subprocess.run(command,shell=True,stdout=subprocess.PIPE,stderr=subprocess.PIPE,encoding="utf-8",timeout=1)
    if ret.returncode == 0:
        print("success:",ret)
    else:
        print("error:",ret)


runcmd(["dir","/b"])#序列参数

Exception in thread Thread-16:
Traceback (most recent call last):
  File "D:\anaconda\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "D:\anaconda\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "D:\anaconda\lib\subprocess.py", line 1063, in _readerthread
    buffer.append(fh.read())
  File "D:\anaconda\lib\codecs.py", line 321, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa5 in position 62: invalid start byte



IndexError: list index out of range

In [21]:
import time
import subprocess

def cmdfun(command):
    subp = subprocess.Popen(command,shell=True,stdout=subprocess.PIPE,stderr=subprocess.PIPE,encoding="utf-8")
    subp.wait(2)
    # 检查进程是否终止
    if subp.poll() == 0:
        # communicate(input,timeout) 和子进程交互，发送和读取数据
        print(subp.communicate()[1])
    else:
        print("失败")

cmdfun("python --version")
cmdfun('dir","/b"])

Python 3.6.5 :: Anaconda custom (64-bit)



Exception in thread Thread-20:
Traceback (most recent call last):
  File "D:\anaconda\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "D:\anaconda\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "D:\anaconda\lib\subprocess.py", line 1063, in _readerthread
    buffer.append(fh.read())
  File "D:\anaconda\lib\codecs.py", line 321, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa5 in position 62: invalid start byte



IndexError: list index out of range

In [16]:
cmdfun(cmd)

失败


# [shutil]复制或者移动文件和目录

你想要复制或移动文件和目录，但是又不想调用 `shell` 命令。

`shutil` 模块有很多便捷的函数可以复制文件和目录。使用起来非常简单

```py
import shutil

# Copy src to dst. (cp src dst)
shutil.copy(src, dst)

# 復制文件的內容以及文件的所有狀態信息。先copyfile後copystat
# Copy files, but preserve metadata (cp -p src dst)
shutil.copy2(src, dst)

# Copy directory tree (cp -R src dst)
shutil.copytree(src, dst)

# Move src to dst (mv src dst)
shutil.move(src, dst)
```

这些函数的参数都是字符串形式的文件或目录名。 底层语义模拟了类似的 `Unix` 命令

如果你只想复制符号链接本身，那么需要指定关键字参数 `follow_symlinks` ,如下：

如果你想保留被复制目录中的符号链接，像这样做：

```py
shutil.copytree(src, dst, symlinks=True)
```

`copytree()` 可以让你在复制过程中选择性的忽略某些文件或目录。 

你可以提供一个忽略函数，接受一个目录名和文件名列表作为输入，返回一个忽略的名称列表。

例如：

```py
def ignore_pyc_files(dirname, filenames):
    return [name in filenames if name.endswith('.pyc')]

shutil.copytree(src, dst, ignore=ignore_pyc_files)
```

由于忽略某种模式的文件名是很常见的，因此一个便捷的函数 `ignore_patterns()` 已经包含在里面了。

例如：

```py
shutil.copytree(src, dst, ignore=shutil.ignore_patterns('*~', '*.pyc'))
```

## 讨论

你通常不会去使用 `shutil.copytree()` 函数来执行系统备份。 

当处理文件名的时候，最好使用 `os.path` 中的函数来确保最大的可移植性（特别是同时要适用于 `Unix` 和 `Windows` ）

```py
>>> filename = '/Users/guido/programs/spam.py'
>>> import os.path
>>> os.path.basename(filename)
'spam.py'
>>> os.path.dirname(filename)
'/Users/guido/programs'
>>> os.path.split(filename)
('/Users/guido/programs', 'spam.py')
>>> os.path.join('/new/dir', os.path.basename(filename))
'/new/dir/spam.py'
>>> os.path.expanduser('~/guido/programs/spam.py')
'/Users/guido/programs/spam.py'
>>>
```

使用 `copytree()` 复制文件夹的一个棘手的问题是对于错误的处理。

例如，在复制过程中，函数可能会碰到损坏的符号链接，因为权限无法访问文件的问题等等。

为了解决这个问题，所有碰到的问题会被收集到一个列表中并打包为一个单独的异常，到了最后再抛出。 

下面是一个例子：

```py
try:
    shutil.copytree(src, dst)
except shutil.Error as e:
    for src, dst, msg in e.args[0]:
         # src is source name
         # dst is destination name
         # msg is error message from exception
         print(dst, src, msg)
```         
         
如果你提供关键字参数 `ignore_dangling_symlinks=True` ， 这时候 `copytree()` 会忽略掉无效符号链接。



## 補充

**Python中shutil模塊的常用文件操作函數用法示例**

https://www.itread01.com/articles/1476166838.html
    

`os` 模塊提供了對目錄或者文件的 新建/刪除/查看 文件屬性

```py
shutil.copyfileobj(fsrc, fdst[, length=16*1024])
# copy文件內容到另一個文件，可以copy指定大小的內容
```  

其中 `fsrc` ， `fdst` 都是文件對象，都需要打開後才能進行復制操作

```py
import shutil
f1=open('name','r')
f2=open('name_copy','w+')
shutil.copyfileobj(f1,f2,length=16*1024)
```

```py
# copyfile調用了copyfileobj
shutil.copyfile(src,dst)

# 僅copy權限，不更改文件內容，組和用戶
shutil.copymode(src,dst)  

# 復制所有的狀態信息，包括權限，組，用戶，時間等
shutil.copystat(src,dst)   

# 復制文件的內容以及權限，先copyfile後copymode
shutil.copy(src,dst) 

# 復制文件的內容以及文件的所有狀態信息。先copyfile後copystat
shutil.copy2(src,dst)

# 遞歸的復制文件內容及狀態信息
shutil.copytree(
    src, 
    dst, 
    symlinks=False, 
    ignore=None, 
    copy_function=copy2,
    ignore_dangling_symlinks=False
)  

# 遞歸地刪除文件
shutil.rmtree(path, ignore_errors=False, onerror=None) 

# 遞歸的移動文件
shutil.move(src, dst)   

make_archive(
    base_name,           # 壓縮打包後的文件名或者路徑名
    format,              # 壓縮或者打包格式    "zip", "tar", "bztar"or "gztar"
    root_dir=None,       # 將哪個目錄或者文件打包（也就是源文件）
    base_dir=None, 
    verbose=0,dry_run=0, 
    owner=None, 
    group=None, 
    logger=None
) 
```